In [1]:
#import dependencies
import pandas as pd
import gmaps
import requests
from config import g_key

In [2]:
#Store CSV into df
city_data_df=pd.read_csv("weather_data/cities.csv")

In [3]:
#Configure gmaps to use your API key
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4) 
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of windspeed
locations = city_data_df[["Lat", "Lng"]]
winds = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=winds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Nha Trang,VN,2021-08-07 20:29:08,12.2500,109.1833,84.20,74,100,5.79
4,4,Odweyne,SO,2021-08-07 20:30:46,9.4092,45.0640,78.76,43,81,17.47
5,5,Sangin,AF,2021-08-07 20:30:47,32.0728,64.8359,85.42,12,0,2.95
7,7,Lompoc,US,2021-08-07 20:30:48,34.6391,-120.4579,82.22,65,1,14.97
14,14,Sinnamary,GF,2021-08-07 20:30:50,5.3833,-52.9500,80.17,84,99,4.97
17,17,Atuona,PF,2021-08-07 20:29:52,-9.8000,-139.0333,76.87,83,88,16.15
18,18,Butaritari,KI,2021-08-07 20:30:51,3.0707,172.7902,82.27,68,12,14.54
25,25,Bengkulu,ID,2021-08-07 20:30:54,-3.8004,102.2655,75.67,91,11,3.89
26,26,Jamestown,US,2021-08-07 20:28:21,42.0970,-79.2353,80.19,74,75,3.00
28,28,Makakilo City,US,2021-08-07 20:30:55,21.3469,-158.0858,82.72,58,75,10.36


In [11]:
preferred_cities_df.count()

City_ID       197
City          197
Country       197
Date          197
Lat           197
Lng           197
Max Temp      197
Humidity      197
Cloudiness    197
Wind Speed    197
dtype: int64

In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Nha Trang,VN,84.20,12.2500,109.1833,
4,Odweyne,SO,78.76,9.4092,45.0640,
5,Sangin,AF,85.42,32.0728,64.8359,
7,Lompoc,US,82.22,34.6391,-120.4579,
14,Sinnamary,GF,80.17,5.3833,-52.9500,
...,...,...,...,...,...,...
552,Tecoanapa,MX,82.20,16.5167,-98.7500,
555,Sassandra,CI,76.23,4.9500,-6.0833,
557,Braganca,BR,78.06,-1.0536,-46.7656,
558,Hare Bay,CA,79.95,48.8499,-54.0148,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotel_df.head(20)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Nha Trang,VN,84.20,12.2500,109.1833,
4,Odweyne,SO,78.76,9.4092,45.0640,
5,Sangin,AF,85.42,32.0728,64.8359,
7,Lompoc,US,82.22,34.6391,-120.4579,
14,Sinnamary,GF,80.17,5.3833,-52.9500,
17,Atuona,PF,76.87,-9.8000,-139.0333,
18,Butaritari,KI,82.27,3.0707,172.7902,
25,Bengkulu,ID,75.67,-3.8004,102.2655,
26,Jamestown,US,80.19,42.0970,-79.2353,
28,Makakilo City,US,82.72,21.3469,-158.0858,


In [22]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))